(interrater-reliability-notebook)=
# L'affidabilità tra giudici

L'affidabilità tra giudici (*Interrater Reliability*) si riferisce alla coerenza o concordanza tra due o più osservatori o giudici che stanno misurando o valutando lo stesso fenomeno o campione. In altre parole, indica quanto siano simili i risultati ottenuti da diversi osservatori o giudici quando si esprimono giudizi o misurazioni simili.

Questo capitolo esamina l'affidabilità tra giudici in due contesti: con due giudici e con giudici molteplici. Nel primo caso, analizzeremo l'accordo nominale e il Kappa di Cohen. Nel secondo caso, utilizzeremo l'ICC, l'ANOVA ad una via e l'ANOVA a due vie. In questo contesto, i modelli a effetti misti si dimostrano fondamentali per gestire disegni con giudici molteplici, catturando variazioni tra i giudici e tra i partecipanti e fornendo una visione completa dell'affidabilità delle valutazioni.

In [1]:
source("_common.R")
suppressPackageStartupMessages(library(lme4))

Nella discussione seguente useremo i dati dello studio [The Reliability and Validity of Speech-Language Pathologists’ Estimations of Intelligibility in Dysarthria](https://www.mdpi.com/2076-3425/12/8/1011/) di Hirsch, Thompson, Kim e Lansford (2022).

In [2]:
slp_dat <- read.csv("https://osf.io/download/p9gqk/")
slp_vas_wide <- slp_dat |>
    select(slpID, Speaker, slp_VAS) |>
    pivot_wider(names_from = slpID, values_from = slp_VAS)
head(slp_vas_wide)

Speaker,slp10,slp11,slp13,slp14,slp15,slp16,slp17,slp18,slp19,⋯,slp21,slp22,slp23,slp25,slp3,slp4,slp5,slp6,slp8,slp9
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AF1,96.28,100.00,96.56,99.30,100.00,12.58,100.00,88.16,52.58,⋯,77.42,73.64,100.00,100.00,37.82,73.24,55.01,100.00,53.94,85.07
AF9,12.32,34.68,0.86,3.52,12.58,0.00,6.69,0.00,7.42,⋯,13.55,0.00,0.00,2.90,0.00,14.51,1.72,NA,6.76,10.28
ALSF6,NA,30.84,5.73,73.94,73.87,4.52,45.13,41.92,26.13,⋯,25.81,8.31,18.46,86.77,0.00,26.20,3.72,NA,9.86,37.75
ALSF7,93.98,96.31,61.32,64.37,100.00,7.74,69.92,88.16,37.42,⋯,59.35,34.10,90.63,53.23,99.71,63.10,56.73,88.83,64.93,53.80
ALSF9,29.51,52.20,43.55,46.76,60.65,23.87,25.21,91.64,37.10,⋯,57.74,29.51,50.14,64.84,0.00,31.13,40.11,16.91,5.92,50.00
ALSM1,96.28,NA,29.23,69.44,99.35,82.26,70.19,79.72,78.39,⋯,83.87,84.24,92.29,100.00,100.00,18.03,58.74,66.48,51.83,60.28


## Due giudici

Selezioniamo unicamente i primi due giudici.

In [3]:
slp_2rater <- slp_vas_wide |>
    select(slp14, slp15)
head(slp_2rater)

slp14,slp15
<dbl>,<dbl>
99.30,100.00
3.52,12.58
73.94,73.87
64.37,100.00
46.76,60.65
69.44,99.35


### Accordo nominale

Per calcolare l'accordo nominale, dobbiamo considerare le valutazioni tra due giudici come esattamente uguali. Se utilizziamo questa definizione, otteniamo:

In [4]:
with(slp_2rater, mean(slp14 == slp15))

[1] 0.1

Se invece rilassiamo leggermente la definizione e diciamo che l'accordo è raggiunto se è lo stesso dopo l'arrotondamento, otteniamo:

In [5]:
slp_2round <- round(slp_2rater / 10)
slp_2round <- lapply(slp_2round, FUN = factor, levels = 0:10)
# Contingency table
table(slp_2round)

     slp15
slp14 0 1 2 3 4 5 6 7 8 9 10
   0  0 1 0 0 0 0 0 0 0 0  0
   1  0 0 0 0 0 0 0 0 0 0  0
   2  0 0 0 0 0 0 0 0 0 0  0
   3  0 0 0 0 0 0 0 0 0 0  0
   4  0 0 0 0 0 0 0 0 0 0  0
   5  0 0 0 0 0 0 1 0 0 0  0
   6  0 0 0 0 0 1 0 0 0 0  2
   7  0 0 0 0 0 0 0 1 0 0  1
   8  0 0 0 0 0 0 0 0 0 2  0
   9  0 0 0 0 0 0 0 0 1 0  2
   10 0 0 0 0 0 0 0 1 0 0  7

In [6]:
p0 <- with(slp_2round, mean(slp14 == slp15))
p0

[1] 0.4

### Kappa di Cohen

L'indice di concordanza Kappa di Cohen è definito nel modo seguente:

$$
\kappa = \frac{p_o - p_c}{1 - p_c},
$$

dove:

- $\kappa$ rappresenta il coefficiente di Kappa di Cohen.
- $p_o$ è l'indice di concordanza osservato tra gli osservatori.
- $p_c$ è l'indice di concordanza attesa per caso.

$p_o$ (indice di concordanza osservato tra gli osservatori):

$$
p_o = \frac{{\text{Numero di accordi osservati}}}{{\text{Numero totale di confronti}}}
$$

In altre parole, $p_o$ è il rapporto tra il numero di volte in cui gli osservatori sono concordi (hanno dato la stessa valutazione) e il numero totale di confronti effettuati.

$p_c$ è l'indice di concordanza attesa per caso:

$$
p_c = \frac{{\sum (\text{Riga i-esima del totale}) \times (\text{Colonna i-esima del totale})}}{{\text{Numero totale di confronti}}^2} 
$$

In altre parole, $p_c$ è il rapporto atteso di concordanza tra gli osservatori nel caso in cui le loro valutazioni siano indipendenti.

Queste formule vengono utilizzate per calcolare il coefficiente di Kappa di Cohen, che misura la concordanza tra osservatori che stanno misurando una variabile categorica, tenendo conto della concordanza casuale attesa. Un valore di $\kappa$ vicino a 1 indica un'alta concordanza, mentre un valore vicino a 0 indica una concordanza casuale e un valore negativo indica una concordanza peggiore di quella casuale.

In [7]:
slp_2tab <- table(slp_2round)
pc <- sum(colSums(slp_2tab) * rowSums(slp_2tab)) / sum(slp_2tab)^2
(kappa <- (p0 - pc) / (1 - pc))

[1] 0.1666667

Svantaggio: Il coefficiente di Kappa tende a essere basso quando i punteggi sono distribuiti in maniera disomogenea (ad esempio, quando la maggior parte dei punteggi appartiene a determinate categorie). Questa situazione corrisponde esattamente a quella descritta sopra.

## Giudici multipli

### Coefficiente $\alpha$

Ciascun giudice è trattato come un "item".

In [8]:
psych::alpha(slp_vas_wide[-1])

Number of categories should be increased  in order to count frequencies. 



Warning message in cor.smooth(r):
“Matrix was not positive definite, smoothing was done”


In smc, smcs > 1 were set to 1.0



In smc, smcs > 1 were set to 1.0



In smc, smcs < 0 were set to .0



...

In smc, smcs > 1 were set to 1.0



In smc, smcs < 0 were set to .0



In smc, smcs > 1 were set to 1.0



In smc, smcs > 1 were set to 1.0




Reliability analysis   
Call: psych::alpha(x = slp_vas_wide[-1])

  raw_alpha std.alpha G6(smc) average_r S/N    ase mean sd median_r
      0.98      0.98    0.99      0.68  45 0.0076   62 25      0.7

    95% confidence boundaries 
         lower alpha upper
Feldt     0.96  0.98  0.99
Duhachek  0.96  0.98  0.99

 Reliability if an item is dropped:
      raw_alpha std.alpha G6(smc) average_r S/N var.r med.r
slp10      0.97      0.98    0.98      0.68  42 0.019  0.70
slp11      0.97      0.98    1.00      0.68  42 0.018  0.70
slp13      0.97      0.98    0.99      0.68  42 0.019  0.70
slp14      0.97      0.98    0.99      0.68  43 0.019  0.71
slp15      0.97      0.98    1.00      0.68  43 0.019  0.71
slp16      0.98      0.98    0.99      0.69  45 0.018  0.71
slp17      0.97      0.98    0.99      0.68  42 0.019  0.69
slp18      0.97      0.98    1.00      0.69  44 0.017  0.71
slp19      0.97      0.98    0.99      0.68  43 0.019  0.71
slp2       0.97      0.98    0.99      0.68  43 

### Coefficiente di correlazione intraclasse

Il Coefficiente di Correlazione Intraclass (ICC) è una misura statistica che descrive quanto le unità all'interno dello stesso gruppo siano simili tra loro. A differenza della maggior parte delle altre misure di correlazione, l'ICC viene utilizzato con dati organizzati in gruppi anziché coppie di osservazioni. Questo lo rende particolarmente adatto quando si desidera valutare la concordanza tra giudici o osservatori che stanno valutando lo stesso gruppo di individui.

L'ICC si basa sul framework del modello ANOVA o del modello a effetti misti. Questi modelli statistici consentono di analizzare le variazioni dei punteggi sia all'interno dei gruppi (ovvero le differenze tra le unità all'interno dello stesso gruppo) che tra i gruppi (ovvero le differenze tra i gruppi stessi). Pertanto, l'ICC tiene conto di due fonti di varianza: la varianza all'interno dei gruppi (varianza delle valutazioni dei giudici all'interno dello stesso gruppo) e la varianza tra i gruppi (varianza delle valutazioni dei giudici tra gruppi diversi). Inoltre, viene considerato l'errore casuale presente nelle valutazioni.

In sintesi, l'ICC è uno strumento utile per valutare l'affidabilità delle valutazioni o misurazioni effettuate da diversi giudici o osservatori su un gruppo di individui. Un ICC più alto indica una maggiore concordanza tra le valutazioni dei giudici, mentre un ICC più basso indica una maggiore variabilità tra le valutazioni. 

Variabili indipendenti: Giudici e individui valutati (ratees)
Variabile dipendente: Punteggi
Fonti di varianza:

1. Ratees (varianza di interesse)
2. Giudici (raters)
3. Interazione tra Ratees e Giudici*
4. Errore casuale*

L'indice di ICC viene calcolato dividendo la varianza dei ratees per la somma della varianza dei ratees e l'errore casuale. Questo indice misura la proporzione della varianza totale dei punteggi attribuita alla variabilità dei ratees, fornendo un'indicazione dell'affidabilità delle valutazioni effettuate dai giudici. Un ICC più alto indica una maggiore concordanza tra i giudici e quindi una maggiore affidabilità delle valutazioni.

Tuttavia, ciò che viene considerato come errore dipende da diverse considerazioni, tra cui:

1. Se il design è "crossed" (cioè tutti i giudici valutano ciascun individuo valutato) o "nested" (diversi gruppi di giudici valutano ciascun individuo valutato). Nel primo caso, l'errore sarà associato alle differenze tra i giudici all'interno dello stesso gruppo di individui valutati, mentre nel secondo caso l'errore sarà relativo alle differenze tra i gruppi di giudici che valutano individui diversi.
2. Se i giudici sono considerati "fixed" (non generalizzabili) o "random" (generalizzabili a una "popolazione di giudici"); questa distinzione diventa rilevante solo quando l'interazione tra giudici e individui valutati può essere stimata. Nel caso in cui i giudici siano considerati "fixed", si presume che i giudici specifici coinvolti nello studio siano l'unica popolazione rilevante, e l'interesse è focalizzato sulla concordanza tra di loro. Nel caso in cui i giudici siano considerati "random", si ammette che i giudici coinvolti nello studio siano solo un campione di una popolazione più ampia di giudici, e l'interesse è sulla concordanza che può essere generalizzata a tale popolazione più vasta.
3. La distinzione tra "consistenza" (decisioni relative; è interessante solo l'ordinamento dei punteggi) e "accordo assoluto" (decisioni assolute; è interessante il valore effettivo del punteggio). Nel contesto della valutazione dei giudici, la consistenza si riferisce al grado in cui i giudici sono concordi nell'ordine di classificazione degli individui valutati, indipendentemente dai valori numerici dei punteggi. D'altra parte, l'accordo assoluto misura il grado di concordanza tra i giudici riguardo ai valori numerici dei punteggi assegnati agli individui valutati.

Queste considerazioni sono fondamentali quando si analizzano e interpretano i risultati dell'Intraclass Correlation (ICC) o di altri metodi di valutazione dell'affidabilità tra giudici o osservatori. La scelta del modello statistico e la definizione dell'errore dipendono da queste varie considerazioni, e la comprensione di questi aspetti è essenziale per una valutazione accurata della concordanza e dell'affidabilità nelle misurazioni effettuate da più giudici o osservatori.

### ANOVA ad una via per disegni nidificati

Consideriamo il caso in cui i dati rappresentano i giudizi assegnati a ciascuna persona da due giudici diversi.

In [9]:
slp_vas_nested <- slp_dat |>
    mutate(SpeakerID = as.numeric(as.factor(Speaker))) |>
    # Select only 10 speakers
    filter(SpeakerID <= 10) |>
    group_by(Speaker) |>
    # Filter specific raters
    filter(row_number() %in% (SpeakerID[1] * 2 - (1:0)))

head(slp_vas_nested)

slpID,Speaker,slp_VAS,slp_VAS.Rel,slp_EST,slp_EST.Rel,SpeakerID
<chr>,<chr>,<dbl>,<dbl>,<int>,<int>,<dbl>
slp10,AF1,96.28,NA,90,NA,1
slp11,AF1,100.00,NA,80,NA,1
slp13,AF9,0.86,NA,10,NA,2
slp14,AF9,3.52,NA,10,NA,2
slp15,ALSF6,73.87,NA,10,NA,3
slp16,ALSF6,4.52,9.03,5,NA,3


Abbiamo un disegno in cui i giudici sono annidati all'interno degli individui valutati. Il disegno "raters nested within ratees" indica che i giudici sono organizzati in gruppi all'interno dei singoli individui valutati. In altre parole, più giudici valutano ciascun individuo, e tali gruppi di giudici sono indipendenti tra loro. Per esempio, potremmo avere vari giudici che valutano diversi studenti in una classe, e i giudici non si sovrappongono tra i diversi studenti valutati. 

Tuttavia, con questa struttura, non possiamo distinguere l'effetto dei giudici (cioè le differenze tra i giudici nelle loro valutazioni) dall'errore casuale (le differenze casuali che possono verificarsi tra le valutazioni dei giudici all'interno dello stesso gruppo). Questo rende difficile capire quanto della variabilità osservata nei punteggi sia dovuto a differenze reali tra i giudici e quanto sia dovuto a fluttuazioni casuali. 

Per affrontare questa situazione, possiamo utilizzare un'analisi della varianza a effetti casuali unici, comunemente nota come "one-way random-effect ANOVA". Questo modello statistico ci consente di stimare la varianza attribuibile alle differenze tra i giudici e la varianza dell'errore casuale. In pratica, il modello considera i giudici come effetti casuali, assumendo che i giudici coinvolti nel nostro studio siano un campione rappresentativo di tutti i possibili giudici che potrebbero essere coinvolti in situazioni simili. Ciò ci permette di stimare la variabilità della valutazione dovuta alla variazione tra i giudici. Questo modello è anche equivalente a un modello multilivello con intercette casuali.

In [10]:
m1 <- lmer(slp_VAS ~ 1 + (1 | Speaker), data = slp_vas_nested)
summary(m1)

Linear mixed model fit by REML ['lmerMod']
Formula: slp_VAS ~ 1 + (1 | Speaker)
   Data: slp_vas_nested

REML criterion at convergence: 179.6

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.50049 -0.77609 -0.05948  0.87584  1.35589 

Random effects:
 Groups   Name        Variance Std.Dev.
 Speaker  (Intercept) 308.9    17.58   
 Residual             816.8    28.58   
Number of obs: 19, groups:  Speaker, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   53.613      8.627   6.214

Estraiamo le fonti di varianza:

In [11]:
vc_m1 <- as.data.frame(VarCorr(m1))
vc_m1

grp,var1,var2,vcov,sdcor
<chr>,<chr>,<chr>,<dbl>,<dbl>
Speaker,(Intercept),NA,308.9062,17.57573
Residual,NA,NA,816.8141,28.57996


Calcoliamo l'ICC per una singola valutazione:

$$
ICC = \frac{\sigma^2_{\text{giudici}}} {\sigma^2_{\text{giudici}} + \sigma^2_E}.
$$

Nella formula, l'ICC misura la proporzione della varianza totale del punteggio `slp_VAS` che è attribuibile alla variazione tra i gruppi dei giudici (Speaker) rispetto alla variazione residua o errore.

In [12]:
vc_m1$vcov[1] / (vc_m1$vcov[1] + vc_m1$vcov[2])

[1] 0.2744076

Calcoliamo l'ICC per la valutazione media:

$$
ICC = \frac{\sigma^2_{\text{giudici}}} {\sigma^2_{\text{giudici}} + \sigma^2_E / k},
$$

dove $k$ è il numero di giudici per ciascun individui valutati (ratee).

In [13]:
vc_m1$vcov[1] / (vc_m1$vcov[1] + vc_m1$vcov[2] / 2)

[1] 0.4306434

### ANOVA a due vie

L'ANOVA a due vie è una tecnica statistica che ci consente di esaminare l'effetto di due o più variabili indipendenti (fattori) sulla variabile dipendente (risposta). Nel nostro caso, stiamo analizzando l'effetto dei giudici (raters) su una variabile dipendente, come ad esempio una scala di valutazione.

Nell'esaminare la consistenza delle misurazioni, ossia quando ci interessa l'ordine di classificazione delle risposte, il bias dei giudici non è considerato un errore. Ciò avviene poiché il bias, ovvero le tendenze o inclinazioni dei giudici nelle loro valutazioni, si applica in modo simile a tutti gli individui valutati e non modifica l'ordine relativo delle valutazioni. In altre parole, se un giudice tende a valutare più alti o più bassi rispetto agli altri giudici, questa tendenza si rifletterà in tutte le valutazioni, ma l'ordine dei valori rimarrà invariato.

D'altra parte, quando valutiamo l'accordo tra i giudici, ossia quando ci interessa la concordanza delle misurazioni in termini di valori assoluti, il bias dei giudici è considerato un errore. Questo accade perché il bias, che potrebbe portare a differenze sistematiche nei punteggi assegnati, influisce direttamente sui valori numerici delle valutazioni. In questo caso, l'effetto del giudice contribuisce alla variabilità totale delle misurazioni, andando oltre la variabilità intrinseca delle risposte.

In sintesi, il trattamento dell'effetto dei giudici come errore o non errore dipende dal contesto in cui vengono effettuate le valutazioni. Nell'analisi di consistenza (decisione relativa), dove l'ordine delle valutazioni è fondamentale, il bias del giudice non è considerato un errore. D'altro canto, nell'analisi di accordo (decisione assoluta), dove i valori numerici esatti delle misurazioni sono cruciali, il bias del giudice è trattato come errore poiché può influenzare direttamente i valori assoluti delle valutazioni.

In [14]:
m2 <- lmer(slp_VAS ~ 1 + (1 | Speaker) + (1 | slpID), data = slp_dat)
summary(m2)

Linear mixed model fit by REML ['lmerMod']
Formula: slp_VAS ~ 1 + (1 | Speaker) + (1 | slpID)
   Data: slp_dat

REML criterion at convergence: 3544.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.3281 -0.5813  0.0862  0.6611  2.5747 

Random effects:
 Groups   Name        Variance Std.Dev.
 slpID    (Intercept) 132.8    11.53   
 Speaker  (Intercept) 585.7    24.20   
 Residual             291.2    17.07   
Number of obs: 403, groups:  slpID, 21; Speaker, 20

Fixed effects:
            Estimate Std. Error t value
(Intercept)   61.882      6.028   10.27

In [15]:
psych::ICC(slp_vas_wide[-1])

Call: psych::ICC(x = slp_vas_wide[-1])

Intraclass correlation coefficients 
                         type  ICC  F df1 df2       p lower bound upper bound
Single_raters_absolute   ICC1 0.58 30  19 400 1.6e-64        0.43        0.75
Single_random_raters     ICC2 0.58 43  19 380 1.0e-82        0.43        0.75
Single_fixed_raters      ICC3 0.67 43  19 380 1.0e-82        0.53        0.81
Average_raters_absolute ICC1k 0.97 30  19 400 1.6e-64        0.94        0.98
Average_random_raters   ICC2k 0.97 43  19 380 1.0e-82        0.94        0.98
Average_fixed_raters    ICC3k 0.98 43  19 380 1.0e-82        0.96        0.99

 Number of subjects = 20     Number of Judges =  21
See the help file for a discussion of the other 4 McGraw and Wong estimates,

Estraiamo le fonti di varianza:

In [16]:
vc_m2 <- as.data.frame(VarCorr(m2))
vc_m2

grp,var1,var2,vcov,sdcor
<chr>,<chr>,<chr>,<dbl>,<dbl>
slpID,(Intercept),NA,132.8257,11.52500
Speaker,(Intercept),NA,585.6568,24.20035
Residual,NA,NA,291.2401,17.06576


ICC for single rating:

$$
ICC = \frac{\sigma^2_{\text{ratee}}} {\sigma^2_{\text{ratee}} + \sigma^2_{\text{rater}} + \sigma^2_E}.
$$

In [17]:
# Note: ratee is in position 2, but may be different in different data sets
vc_m2$vcov[2] / (vc_m2$vcov[1] + vc_m2$vcov[2] + vc_m2$vcov[3])

[1] 0.5800175

Consistency:

$$
ICC = \frac{\sigma^2_{\text{ratee}}} {\sigma^2_{\text{ratee}} + \sigma^2_E}.
$$

In [18]:
# Note: ratee is in position 2, but may be different in different data sets
vc_m2$vcov[2] / (vc_m2$vcov[2] + vc_m2$vcov[3])

[1] 0.6678742

ICC for average rating:

$$
Agreement = \frac{\sigma^2_{\text{ratee}}} {\sigma^2_{\text{ratee}} + (\sigma^2_{\text{rater}} + \sigma^2_E) / k}.
$$

In [19]:
# Note: ratee is in position 2, but may be different in different data sets
vc_m2$vcov[2] / (vc_m2$vcov[2] + (vc_m2$vcov[1] + vc_m2$vcov[3]) / 21)

[1] 0.966669

$$
Consistency = \frac{\sigma^2_{\text{ratee}}} {\sigma^2_{\text{ratee}} + \sigma^2_E / k}.
$$

In [20]:
vc_m2$vcov[2] / (vc_m2$vcov[2] + vc_m2$vcov[3] / 21)

[1] 0.9768674

Tutti questi risultati possono essere ottenuti con la funzione `psych::ICC()`. I dati devono essere in formato wide.

In [21]:
psych::ICC(slp_vas_wide[-1])

Call: psych::ICC(x = slp_vas_wide[-1])

Intraclass correlation coefficients 
                         type  ICC  F df1 df2       p lower bound upper bound
Single_raters_absolute   ICC1 0.58 30  19 400 1.6e-64        0.43        0.75
Single_random_raters     ICC2 0.58 43  19 380 1.0e-82        0.43        0.75
Single_fixed_raters      ICC3 0.67 43  19 380 1.0e-82        0.53        0.81
Average_raters_absolute ICC1k 0.97 30  19 400 1.6e-64        0.94        0.98
Average_random_raters   ICC2k 0.97 43  19 380 1.0e-82        0.94        0.98
Average_fixed_raters    ICC3k 0.98 43  19 380 1.0e-82        0.96        0.99

 Number of subjects = 20     Number of Judges =  21
See the help file for a discussion of the other 4 McGraw and Wong estimates,

I tipi di Intraclass Correlation Coefficients (ICC) riportati nell'output hanno i seguenti significati:

1. `Single_raters_absolute`, ICC(1, 1): Questo tipo di ICC misura l'affidabilità delle valutazioni effettuate da singoli giudici su ciascun individuo valutato in un'ANOVA unidirezionale. Si assume che ogni individuo sia valutato da un gruppo di giudici diverso. L'ICC calcolato indica la concordanza tra i giudici nelle loro valutazioni per ciascun individuo, ma non tiene conto dell'accordo tra i giudici stessi.

2. `Average_raters_absolute`, ICC(1, k): Questo tipo di ICC misura l'affidabilità delle valutazioni medie ottenute da gruppi diversi di giudici per ciascun individuo valutato in un'ANOVA unidirezionale. Si assume che ogni individuo sia valutato da un gruppo di giudici diverso. L'ICC calcolato fornisce un'indicazione sulla concordanza tra i giudici considerando le medie delle valutazioni ottenute da gruppi diversi di giudici.

3. `Single_random_raters`, ICC(2, 1): Questo tipo di ICC valuta l'accordo tra i giudici nelle loro valutazioni per ciascun individuo in un'ANOVA bidirezionale. Il termine "random" indica che i giudici sono scelti casualmente da una popolazione più ampia di giudici. In altre parole, si considera che i giudici coinvolti nel campione siano solo una selezione casuale e rappresentativa della popolazione completa di giudici. L'ANOVA bidirezionale tiene conto delle valutazioni multiple ottenute da ciascun individuo valutato dai diversi giudici.

4. `Average_random_absolute`, ICC(2, k): Questo tipo di ICC misura l'accordo tra i giudici considerando le valutazioni medie ottenute da gruppi diversi di giudici per ciascun individuo in un'ANOVA bidirezionale. Anche qui, si assume che i giudici siano scelti casualmente dalla popolazione più ampia di giudici. L'ANOVA bidirezionale tiene conto delle valutazioni multiple ottenute dai diversi giudici.

5. `Single_fixed_raters`, ICC(3, 1): Questo tipo di ICC misura la consistenza tra i giudici nelle loro valutazioni per ciascun individuo in un'ANOVA bidirezionale. Il termine "fixed" indica che i giudici sono considerati una parte fissa e specifica del campione. L'ANOVA bidirezionale tiene conto delle valutazioni multiple ottenute da ciascun individuo valutato dai diversi giudici.

6. `Average_fixed_absolute`, ICC(3, k): Questo tipo di ICC misura la consistenza tra i giudici considerando le valutazioni medie ottenute da gruppi diversi di giudici per ciascun individuo in un'ANOVA bidirezionale. Anche qui, si considera che i giudici siano una parte fissa e specifica del campione. L'ANOVA bidirezionale tiene conto delle valutazioni multiple ottenute dai diversi giudici.

In questo esempio, ICC(1, 1) e ICC(1, k) non sono rilevanti, poiché si tratta di un design incrociato (non nidificato).

## Considerazioni conclusive

In conclusione, in questo capitolo abbiamo esaminato l'affidabilità tra giudici in due contesti distinti: quello che coinvolge soltanto due giudici e quello con giudici molteplici.

Nel contesto con due giudici, abbiamo affrontato l'accordo nominale e calcolato il coefficiente di Kappa di Cohen. L'accordo nominale si basa sull'assunzione che le valutazioni tra i due giudici siano esattamente uguali, o uguali dopo un arrotondamento. Al contrario, il Kappa di Cohen è una misura più appropriata dell'accordo, poiché tiene conto anche dell'errore di misurazione. 

Nel contesto con giudici molteplici, abbiamo utilizzato tecniche più sofisticate per affrontare la complessità dell'interazione tra più giudici. Qui abbiamo adottato l'Intraclass Correlation Coefficient (ICC), l'ANOVA ad una via e l'ANOVA a due vie.

L'ICC è stato utilizzato per misurare la consistenza delle valutazioni tra giudici, considerando sia la variabilità tra i giudici sia la variabilità all'interno di ciascun giudice. Questa metodologia ci ha fornito una stima della proporzione di varianza attribuibile alle differenze tra giudici rispetto alla varianza totale delle valutazioni. È importante notare che l'ICC è strettamente correlato con i modelli a effetti misti, poiché questi modelli sono in grado di affrontare i disegni con giudici molteplici e catturare sia le variazioni tra giudici che le variazioni all'interno di ciascun giudice.

L'ANOVA ad una via è stato utilizzata quando le valutazioni dei giudici erano raggruppate in un unico fattore, ad esempio le valutazioni di diversi giudici su un gruppo di partecipanti. Questo ci ha permesso di quantificare le differenze tra i giudici nelle loro valutazioni complessive. Nei modelli a effetti misti, un'analisi simile può essere eseguita includendo il giudice come effetto casuale, consentendo di considerare sia le differenze tra i giudici sia le differenze tra i partecipanti.

Infine, l'ANOVA a due vie è stato adottata quando le valutazioni dei giudici sono state analizzate considerando due fattori, ad esempio giudici diversi che valutano diversi partecipanti. Questo ci ha consentito di esaminare l'effetto sia dei giudici che dei partecipanti sulle valutazioni complessive. I modelli a effetti misti possono essere estesi per gestire questa configurazione, includendo sia il giudice che il partecipante come effetti casuali, permettendo una comprensione completa delle interazioni tra i due fattori.

In conclusione, abbiamo esplorato diverse metodologie per valutare l'affidabilità tra giudici in diversi contesti, sia con due giudici che con giudici molteplici. I modelli a effetti misti si rivelano preziosi strumenti statistici per analizzare i disegni con giudici molteplici, poiché ci permettono di catturare le variazioni tra i giudici e tra i partecipanti, fornendo una visione globale dell'affidabilità delle valutazioni. 